# Analisis de archivos ab1 y obtener secuencias consenso
información  
https://biopython.org/wiki/SeqIO  
http://biopython.org/wiki/ABI_traces

In [ ]:
from Bio import SeqIO, SeqRecord, Seq, AlignIO, pairwise2
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC, generic_dna
from collections import defaultdict
import matplotlib.pyplot as plt
from io import StringIO
from Bio.Align import AlignInfo
from Bio.SubsMat.MatrixInfo import blosum62



In [ ]:
def consenso2(sec1, sec2):
    if len(sec1)!=len(sec2):
        print("las secuencias no son de igual tamaño")
        return
    n=0
    secuencia= ""
    nucleotido=""
    for row in range(len(sec1)):
        n+=1
        if sec1[row]== sec2[row]:
            secuencia+= sec1[row]
            #print (sec1[row], end = "")
        else:
            if sec1[row]=="-" and sec2[row]!="-":
                nucleotido=sec2[row].lower()
            elif sec1[row]!="-" and sec2[row]=="-":
                nucleotido=sec1[row].lower()
            elif sec1[row]!= sec2[row]:
                if (sec1[row]=="A" and sec2[row]== "G") or (sec1[row]=="G" and sec2[row]== "A"):
                    nucleotido="R"
                elif (sec1[row]=="C" and sec2[row]== "T") or (sec1[row]=="T" and sec2[row]== "C"):
                    nucleotido="Y"
                elif (sec1[row]=="G" and sec2[row]== "C") or (sec1[row]=="C" and sec2[row]== "G"):
                    nucleotido="S"
                elif (sec1[row]=="A" and sec2[row]== "T") or (sec1[row]=="T" and sec2[row]== "A"):
                    nucleotido="W"
                elif (sec1[row]=="G" and sec2[row]== "T") or (sec1[row]=="T" and sec2[row]== "T"):
                    nucleotido="K"
                elif (sec1[row]=="A" and sec2[row]== "C") or (sec1[row]=="C" and sec2[row]== "A"):
                    nucleotido="M"
            else:
                print ("hay dos errores, posición",row ,sec1[row], sec2[row])
                secuencia += nucleotido
            #print ( nucleotido, end = "")
            secuencia+= nucleotido

        if n==93:
            #print()
            n=0
    return (secuencia)

In [ ]:
def graficado(x1,x2):
    plt.figure(figsize=(16,4))
    plt.plot(trace['DATA9'], color='blue')
    plt.plot(trace['DATA10'], color='red')
    plt.plot(trace['DATA11'], color='green')
    plt.plot(trace['DATA12'], color='yellow')
    plt.xlim(x1,x2)  # se utiliza valores de 1000 como maximo
    plt.show()

In [ ]:
cd /Users/migueldelrio/Documents/2019Cicese/2019uam/cursosuam2019/pdi_molecular/secuencias/seq

In [ ]:
ls -lh *.ab1

# Se trabajará con las secuencias
`P1a-16S-F.ab1`  
`P1a-16S-R.ab1`
# Secuencias con el iniciador sentido (forward)

In [ ]:
recordtrimmed_F = SeqIO.read('P1a-16S-F.ab1', 'abi-trim')
recordtrimmed_R = SeqIO.read('P1a-16S-R.ab1', 'abi-trim')

In [ ]:
alignments = pairwise2.align.globalds(recordtrimmed_F.seq,recordtrimmed_R.seq.reverse_complement(), 
                                      blosum62, -10, -0.5)

In [ ]:
for row in range(len(alignments)):
    print ("Alineamiento", row)
    print(pairwise2.format_alignment(*alignments[row]), "\n")

# Seleccionar alineamiento en 
`consenso2(alignments[x][0], alignments[x][1])`  
donde `x` es la secuencia seleccionada

In [ ]:
secuencia = consenso2(alignments[1][0], alignments[1][1])
print(secuencia)

# Las letras minúsculas indican que no hay nucleótidos en una de las dos secuencias
# Se utiliza el código IUPAC para designar variantes 

|Letra|nucleótidos|
|-----|-----------|
|R|A or G|
|Y|C or T|
|S|G or C|
|W|A or T|
|K|G or T|
|M|A or C|

In [ ]:
secuencia_fas = SeqRecord(Seq(secuencia.upper()), id="P1a-16S", description="P1a-16S" )

## Guardando la secuencia consenso

In [ ]:
SeqIO.write(secuencia_fas, 'P1a-16S_con.fasta', 'fasta')

In [ ]:
pwd

In [ ]:
%%bash
export BLASTDB=/Volumes/Respaldo_completo/Escritorio/bigdata/nt/
cd ~/Documents/2019Cicese/2019uam/cursosuam2019/pdi_molecular/secuencias/seq
date
time blastn \
-query P1a-16S_con.fasta \
-db /Volumes/Respaldo_completo/Escritorio/bigdata/nt/nt \
-out P1a-16S_con.tab \
-evalue 1E-6 \
-max_target_seqs 1 \
-num_threads 1 \
-outfmt "6 std sskingdoms stitle staxids sscinames scomnames sblastnames" 

